In [2]:
import psycopg2

def create_DB(cur):
    cur.execute('''
        CREATE TABLE IF NOT EXISTS clients (
            id SERIAL PRIMARY KEY,
            name VARCHAR(255) NOT NULL,
            surname VARCHAR(255) NOT NULL,
            email VARCHAR(255) NOT NULL);
            ''')
    cur.execute('''
        CREATE TABLE IF NOT EXISTS phones (
            phone_number VARCHAR(11) PRIMARY KEY,
            client_id INTEGER REFERENCES clients(id)           
            );
            ''')
    return
 



In [3]:
def test(cur):
    
    cur.execute('''
        SELECT * FROM clients;
        ''')
    print('fetchall', cur.fetchall())
    cur.execute('''
        SELECT * FROM phones;
        ''')
    print('fetchall', cur.fetchall())
    return

In [4]:
def add_phone(cur, phone_number, client_id):
    cur.execute('''
        insert into phones (phone_number, client_id)
        values (%s, %s);
        ''', (phone_number, client_id))
    return client_id

In [5]:
def add_client(cur, name=None, surname=None, email=None, phone_number=None):
    cur.execute('''
        INSERT INTO clients (name, surname, email)
        VALUES (%s, %s, %s);
        ''', (name, surname, email))
    
    cur.execute("""
        SELECT id from clients
        ORDER BY id DESC
        LIMIT 1
        """)
    
    id = cur.fetchone()[0]
    if phone_number is None:
        return id
    else: 
        add_phone(cur, phone_number, id)
        return id
    
    

In [6]:
def update_client(cur, id, name=None, surname=None, email=None):
    cur.execute(""" 
        SELECT * from clients
        WHERE id = %s
        """, (id, ))
    info = cur.fetchone()
    
    if name is None:
        name = info[1]
    if surname is None:
        surname = info[2]
    if email is None:
        email = info[3]
    cur.execute("""
        UPDATE clients
        SET name = %s, surname = %s, email =%s 
        where id = %s
        """, (name, surname, email, id)) 
    return

In [7]:
def delete_phone(cur, phone_number):
    cur.execute("""
        DELETE FROM phones 
        WHERE phone_number = %s
        """, (phone_number, ))
    return phone_number

In [8]:
def delete_client(cur, id):
    cur.execute("""
        DELETE FROM phones
        WHERE client_id = %s
        """, (id, ))
    cur.execute("""
        DELETE FROM clients 
        WHERE id = %s
       """, (id,))
    return id

In [16]:
def find_client(cur, name=None, surname=None, email=None, phone_number=None):
    if name is None:
        name = '%'
    else:
        name = '%' + name + '%'
    if surname is None:
        surname = '%'
    else:
        surname = '%' + surname + '%'
    if email is None:
        email = '%'
    else:
        email = '%' + email + '%'
    if phone_number is None:
        cur.execute("""
            SELECT c.id, c.name, c.surname, c.email, p.phone_number FROM clients c
            LEFT JOIN phones p ON c.id = p.client_id
            WHERE c.name LIKE %s AND c.surname LIKE %s
            AND c.email LIKE %s
            """, (name, surname, email))
    else:
        cur.execute("""
             SELECT c.id, c.name, c.surname, c.email, p.phone_number FROM clients c
             LEFT JOIN phones p ON c.id = p.client_id
             WHERE c.name LIKE %s AND c.surname LIKE %s
             AND c.email LIKE %s AND p.phone_number like %s
             """, (name, surname, email, phone_number))
    
    return cur.fetchall()

        


In [10]:
def delete_DB(cur):
    # cur.execute(""" 
    #     SELECT * from clients
    #     """)
    # i = cur.fetchone()
    # if i is not None:
        cur.execute("""
        DROP TABLE clients, phones CASCADE;
        """)
   

In [11]:
#удаляем БД 
if __name__ == '__main__':
    with psycopg2.connect(database="netology_DB", user="postgres", password="1") as conn:
        with conn.cursor() as curs:
            #удаляем БД
            delete_DB(curs)

In [23]:
#Создаем и заполняем БД
if __name__ == '__main__':
    with psycopg2.connect(database="netology_DB", user="postgres", password="1") as conn:
        with conn.cursor() as curs:
            delete_DB(curs)
            
            create_DB(curs)
            print("База данных создана")
            #test(curs)
            add_client(curs, "name1", "surname1", "11@mail.com", "89999999991")
            add_client(curs, "name2", "surname2", "22@mail.com", "89999999992")
            add_client(curs, "name8", "surname8", "88@mail.com")
            add_client(curs, "name9", "surname9", "99@mail.com")
            add_client(curs, "name10", "surname10", "1010@mail.com", "89999999910")
            #test(curs)
            add_phone(curs, "89999999111", 1)
            #test(curs)
            update_client(curs, 1, "name1", "surname1", "11111@gmail.com")
            #test(curs)
            delete_phone(curs, "89999999991")
            test(curs)
            #delete_client(curs, 1)
            #test(curs)
            print(find_client(curs, 'name9'))
            print(find_client(curs, None, None, '88@mail.com'))
            print(find_client(curs, None, None, None, '89999999992'))

База данных создана
fetchall [(2, 'name2', 'surname2', '22@mail.com'), (3, 'name8', 'surname8', '88@mail.com'), (4, 'name9', 'surname9', '99@mail.com'), (5, 'name10', 'surname10', '1010@mail.com'), (1, 'name1', 'surname1', '11111@gmail.com')]
fetchall [('89999999992', 2), ('89999999910', 5), ('89999999111', 1)]
[(4, 'name9', 'surname9', '99@mail.com', None)]
[(3, 'name8', 'surname8', '88@mail.com', None)]
[(2, 'name2', 'surname2', '22@mail.com', '89999999992')]
